In [14]:
import numpy
import iris
from iris.experimental.regrid import regrid_weighted_curvilinear_to_rectilinear

In [8]:
data_file = '/g/data/r87/dbi599/DRSv2/CMIP5/NorESM1-M/historical/yr/ocean/r1i1p1/ohc/latest/ohc_Oyr_NorESM1-M_historical_r1i1p1_185001-185312.nc'
area_file = '/g/data/ua6/DRSv2/CMIP5/NorESM1-M/historical/fx/ocean/r0i0p0/areacello/latest/areacello_fx_NorESM1-M_historical_r0i0p0.nc'

## Iris

In [20]:
data_cube = iris.load_cube(data_file, 'ocean heat content')[0]
print(data_cube)

ocean heat content / (J)            (cell index along second dimension: 384; cell index along first dimension: 320)
     Dimension coordinates:
          cell index along second dimension                           x                                      -
          cell index along first dimension                            -                                      x
     Auxiliary coordinates:
          latitude                                                    x                                      x
          longitude                                                   x                                      x
     Scalar coordinates:
          time: 1850-07-02 12:00:00, bound=(1850-01-01 00:00:00, 1851-01-01 00:00:00)
     Attributes:
          Conventions: CF-1.5
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_NorESM1-M_historical_r0i0p0.nc...
          branch_time: 255135.0
          cmor_version: 2.5.9
          conta

In [10]:
area_cube = iris.load_cube(area_file)
area_cube

<iris 'Cube' of cell_area / (m2) (cell index along second dimension: 384; cell index along first dimension: 320)>

In [12]:
def make_grid(lat_values, lon_values):
    """Make a dummy cube with desired grid."""
       
    latitude = iris.coords.DimCoord(lat_values,
                                    standard_name='latitude',
                                    units='degrees_north',
                                    coord_system=iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS))
    longitude = iris.coords.DimCoord(lon_values,                    
                                     standard_name='longitude',
                                     units='degrees_east',
                                     coord_system=iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS))

    dummy_data = numpy.zeros((len(lat_values), len(lon_values)))
    new_cube = iris.cube.Cube(dummy_data, dim_coords_and_dims=[(latitude, 0), (longitude, 1)])

    new_cube.coord('longitude').guess_bounds()
    new_cube.coord('latitude').guess_bounds()

    return new_cube

In [24]:
grid_res = 1.5
lats = numpy.arange(-90, 90.01, grid_res)
lons = numpy.arange(0, 360, grid_res)
target_grid_cube = make_grid(lats, lons)
target_grid_cube

<iris 'Cube' of unknown / (unknown) (latitude: 121; longitude: 240)>

In [25]:
data_cube.coord(axis='x').coord_system = iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS)
data_cube.coord(axis='y').coord_system = iris.coord_systems.GeogCS(iris.fileformats.pp.EARTH_RADIUS)
regridded_cube = regrid_weighted_curvilinear_to_rectilinear(data_cube, area_cube.data, target_grid_cube)

In [26]:
data_cube.data.sum()

1.5558972140441873e+27

In [27]:
regridded_cube.data.sum()

3.3695111439001594e+26